In [27]:
import pandas as pd
import os

# 기본 경로 설정
base_path = '/Users/apstat/Desktop/연구/멀티모달 밸런싱/데이터'

# 데이터프레임 리스트
all_dfs = []

# 1부터 40까지 순서대로 시도
for i in range(1, 41):
    # 파일명 패턴 (0을 붙여서 Sess01.csv, Sess02.csv 형태 맞추기)
    file_name = f"Sess{i:02d}.csv"
    file_path = os.path.join(base_path, file_name)
    
    # 파일 존재 여부 확인 후 처리
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            all_dfs.append(df)
            print(f"{file_name} 로드 완료 ({len(df)}행)")
        except Exception as e:
            print(f"[오류] {file_name} 읽기 실패: {e}")
    else:
        print(f"[경고] {file_name} 파일이 존재하지 않습니다.")

# 병합 실행
if all_dfs:
    df_all = pd.concat(all_dfs, ignore_index=True)
    print("\n✅ 데이터 병합 완료!")
    print(f"총 {len(df_all)}개의 행이 로드되었습니다.")
    if 'Emotion' in df_all.columns:
        print(f"Emotion 클래스: {df_all['Emotion'].unique()}")
else:
    print("\n❌ 병합할 데이터가 없습니다. 파일 경로를 확인하세요.")


Sess01.csv 로드 완료 (7628행)
Sess02.csv 로드 완료 (7417행)
Sess03.csv 로드 완료 (6681행)
Sess04.csv 로드 완료 (7552행)
Sess05.csv 로드 완료 (6565행)
Sess06.csv 로드 완료 (7433행)
Sess07.csv 로드 완료 (7284행)
Sess08.csv 로드 완료 (7382행)
Sess09.csv 로드 완료 (7504행)
Sess10.csv 로드 완료 (6528행)
Sess11.csv 로드 완료 (6794행)
Sess12.csv 로드 완료 (7465행)
Sess13.csv 로드 완료 (7372행)
Sess14.csv 로드 완료 (6244행)
Sess15.csv 로드 완료 (7104행)
Sess16.csv 로드 완료 (7861행)
Sess17.csv 로드 완료 (7601행)
Sess18.csv 로드 완료 (7006행)
Sess19.csv 로드 완료 (8393행)
Sess20.csv 로드 완료 (7036행)
Sess21.csv 로드 완료 (7406행)
Sess22.csv 로드 완료 (7966행)
Sess23.csv 로드 완료 (8164행)
Sess24.csv 로드 완료 (8441행)
Sess25.csv 로드 완료 (8448행)
Sess26.csv 로드 완료 (7132행)
Sess27.csv 로드 완료 (7878행)
Sess28.csv 로드 완료 (6762행)
Sess29.csv 로드 완료 (7324행)
Sess30.csv 로드 완료 (8117행)
Sess31.csv 로드 완료 (7346행)
Sess32.csv 로드 완료 (8631행)
Sess33.csv 로드 완료 (7583행)
Sess34.csv 로드 완료 (7742행)
Sess35.csv 로드 완료 (7751행)
Sess36.csv 로드 완료 (7440행)
Sess37.csv 로드 완료 (8035행)
Sess38.csv 로드 완료 (7565행)
Sess39.csv 로드 완료 (7827행)
Sess40.csv 로드 완료 (8600행)


In [28]:
df_all.head()

,Segment_ID,Time,EDA,TEMP,Emotion,Valence,Arousal
0,Sess01_script01_User002M_001,0.25,2.856493,34.81,neutral,3.4,2.9
1,Sess01_script01_User002M_001,0.50,2.788578,34.81,neutral,3.4,2.9
2,Sess01_script01_User002M_001,0.75,2.678377,34.81,neutral,3.4,2.9
3,Sess01_script01_User002M_001,1.00,2.652749,34.79,neutral,3.4,2.9
4,Sess01_script01_User002M_001,1.25,2.645061,34.79,neutral,3.4,2.9


In [29]:
len(df_all)

301008

In [30]:
# --- 2. 특징 공학 (Feature Engineering) ---
# (df_all이 1단계에서 준비되었다고 가정)

# 2a. 시계열 특징 (EDA, TEMP) 요약
ts_features = df_all.groupby('Segment_ID')[['EDA', 'TEMP']].agg(
    ['mean', 'std', 'min', 'max']
)
ts_features.columns = ['_'.join(col) for col in ts_features.columns]

# 2b. 정적 특징 (Valence, Arousal, Emotion) 추출
static_features = df_all.groupby('Segment_ID')[['Valence', 'Arousal', 'Emotion']].first()

# 2c. [수정됨] 특징 결합
# 'on=' 대신 'index=' 기준으로 병합
feature_df = pd.merge(
    ts_features, 
    static_features, 
    left_index=True,  # 왼쪽 DataFrame의 인덱스(Segment_ID) 사용
    right_index=True, # 오른쪽 DataFrame의 인덱스(Segment_ID) 사용
    how='inner'
)

# 2d. [추가됨] 인덱스를 다시 열로 변환
# 'Segment_ID'를 인덱스에서 일반 열(Column)로 되돌립니다.
# 3단계의 .drop()이 작동하려면 이 과정이 필수입니다.
feature_df = feature_df.reset_index()

# 2e. NaN 값 처리
# feature_df = feature_df.fillna(0)

In [31]:
len(feature_df)

12763

In [32]:
feature_df['Segment_ID'].unique()

array(['Sess01_script01_User001F_001', 'Sess01_script01_User001F_002',
       'Sess01_script01_User001F_003', ...,
       'Sess40_script06_User080F_023', 'Sess40_script06_User080F_024',
       'Sess40_script06_User080F_025'], shape=(12763,), dtype=object)

In [33]:
# --- 3. MLP 데이터 준비 (6:2:2 분리) ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 3a. X (특징)와 y (타겟) 분리
X = feature_df.drop(columns=['Segment_ID', 'Emotion'])
y = feature_df['Emotion'].astype(str)

# 3b. 1차 분리: Train (60%) vs Temp (40%)
# stratify=y : 7개 감정 클래스 비율을 유지
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.4,  # 1 - 0.6 = 0.4
    random_state=42, 
    stratify=y
)

# 3c. 2차 분리: Temp (40%) -> Validation (20%) vs Test (20%)
# test_size=0.5 : 40%의 50% = 20%
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5, # 0.4 * 0.5 = 0.2
    random_state=42, 
    stratify=y_temp # y_temp로 비율 유지
)

print(f"Total samples: {len(X)}")
print(f"Train samples: {len(X_train)} (60%)")
print(f"Validation samples: {len(X_val)} (20%)")
print(f"Test samples: {len(X_test)} (20%)")

# 3d. 특징 스케일링 (매우 중요!)
scaler = StandardScaler()

# 60%의 Train 데이터로만 '학습(fit)'
X_train_scaled = scaler.fit_transform(X_train)

# Validation과 Test 데이터는 '변환(transform)'만 수행
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\nData preparation and scaling complete.")

Total samples: 12763
Train samples: 7657 (60%)
Validation samples: 2553 (20%)
Test samples: 2553 (20%)

Data preparation and scaling complete.


In [34]:
# --- 4. MLP 모델 학습 및 평가 ---
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

print("\nStarting MLP model training...")

# 4a. MLP 모델 정의
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    max_iter=500, 
    random_state=42,
    activation='relu',
    solver='adam',
    # === Validation Set을 사용한 조기 종료 (튜닝 예시) ===
    # validation_fraction=0.1 (여기서는 사용하지 않음. 수동 분리함)
    # n_iter_no_change=10 # 성능 개선이 10번 없으면 중지
)

# 4b. 모델 학습 (Train set 사용)
# 스케일링된 60% 학습 데이터로 학습
mlp_model.fit(X_train_scaled, y_train)

print("MLP model training complete.")

# 4c. 모델 평가 (Validation set 사용 - 튜닝 단계)
# Validation set은 모델의 하이퍼파라미터(예: 은닉층 개수)를
# 튜닝하기 위해 사용합니다. (예: (100,)이 좋은지 (50, 20)이 좋은지 비교)
y_pred_val = mlp_model.predict(X_val_scaled)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"\n--- Model Performance (Validation Set) ---")
print(f"Validation Accuracy: {accuracy_val:.4f}")
print("(참고용 Validation Report)\n", classification_report(y_val, y_pred_val, digits=4))


Starting MLP model training...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


MLP model training complete.

--- Model Performance (Validation Set) ---
Validation Accuracy: 0.9127
(참고용 Validation Report)
               precision    recall  f1-score   support

       angry     0.6667    0.3571    0.4651        28
     disgust     0.5000    0.1667    0.2500        12
        fear     0.0000    0.0000    0.0000         9
       happy     0.7700    0.6581    0.7097       234
     neutral     0.9276    0.9774    0.9518      2214
         sad     0.0000    0.0000    0.0000        24
    surprise     0.0000    0.0000    0.0000        32

    accuracy                         0.9127      2553
   macro avg     0.4092    0.3085    0.3395      2553
weighted avg     0.8846    0.9127    0.8968      2553



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:

In [36]:
# 4d. 최종 성능 평가 (Test set 사용)
# 모델 튜닝이 모두 끝났다고 가정하고, 
# '단 한 번' Test set을 사용하여 최종 성능을 평가합니다.
y_pred_test = mlp_model.predict(X_test_scaled)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"\n--- FINAL Model Evaluation (Test Set) ---")
print(f"Test Accuracy: {accuracy_test:.4f}")
print("\nFinal Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test, digits=4, zero_division=0))
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# MLP의 수렴 경고를 끔 (선택 사항)
# from sklearn.exceptions import ConvergenceWarning00
# warnings.filterwarnings("ignore", category=ConvergenceWarning)



--- FINAL Model Evaluation (Test Set) ---
Test Accuracy: 0.9138

Final Classification Report (Test Set):
              precision    recall  f1-score   support

       angry     0.4500    0.3103    0.3673        29
     disgust     0.0000    0.0000    0.0000        12
        fear     0.0000    0.0000    0.0000         8
       happy     0.7833    0.6766    0.7260       235
     neutral     0.9296    0.9779    0.9531      2214
         sad     0.0000    0.0000    0.0000        24
    surprise     0.0000    0.0000    0.0000        31

    accuracy                         0.9138      2553
   macro avg     0.3090    0.2807    0.2924      2553
weighted avg     0.8834    0.9138    0.8976      2553



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [39]:
y

0        neutral
1        neutral
2        neutral
3          happy
4        neutral
          ...   
12758    neutral
12759    neutral
12760    neutral
12761    neutral
12762    neutral
Name: Emotion, Length: 12763, dtype: object